# Extract all ingredients from all recipes and Net Carb Calculation Test

In [1]:
import os
import re
import yaml

def extract_recipe_data(file_path, file_name):
    recipe_data = ''
    with open(file_path, 'r') as file:
        content = file.read()

    # Extract the recipe data section
    if 'parse_recipe()' in content:
        recipe_data = content.split('parse_recipe()}}')[0].split('"""')[1].strip()
        #print(file_name)
        recipe_data = recipe_data + "\n\n" + f':FileName {file_name}'
        return recipe_data
    else:
        return None

def process_directory(directory):
    collated_recipe_data = []
    for root, dirs, files in os.walk(directory):
        for file_name in files:
            if file_name.endswith('.md'):
                file_path = os.path.join(root, file_name)
                recipe_name = ' '.join(file_name.split('.')[0].split('_')[:3]).capitalize()
                print(f'[{recipe_name}]({directory}/{file_name})')
                recipe_data = extract_recipe_data(file_path, file_name)
                
                if recipe_data:
                    #print(f"Recipe data from {file_path}: {file_name}")
                    #print(recipe_data)
                    #print("-" * 40)
                    collated_recipe_data.append(recipe_data)
    return collated_recipe_data
# Example usage
directory_path = './docs/Recipes'
results = ''
results = process_directory(directory_path)
#result = extract_recipe_data(directory_path)
print(results[1])

[00-prepare-to-beat](./docs/Recipes/00-prepare-to-beat.md)
[Index](./docs/Recipes/index.md)
[Index](./docs/Recipes/index.md)
[Recipe 1 tava](./docs/Recipes/recipe_1_tava_roti.md)
[Recipe 2 wheat](./docs/Recipes/recipe_2_wheat_gluten_bread.md)
[Recipe 3 tandoori](./docs/Recipes/recipe_3_tandoori_roti.md)
[Recipe 4 naan](./docs/Recipes/recipe_4_naan_bread.md)
[Index](./docs/Recipes/index.md)
[Recipe 001 flaxseed](./docs/Recipes/recipe_001_flaxseed_porridge.md)
[Recipe 002 almond](./docs/Recipes/recipe_002_almond_bread.md)
[Recipe 003 keto](./docs/Recipes/recipe_003_keto_dhokla.md)
[Recipe 004 scrambled](./docs/Recipes/recipe_004_scrambled_tofu.md)
[Recipe 005 keto](./docs/Recipes/recipe_005_keto_upma.md)
[Index](./docs/Recipes/index.md)
[Recipe 001 peanut](./docs/Recipes/recipe_001_peanut_mugcake.md)
[Recipe 002 bottlegourd](./docs/Recipes/recipe_002_bottlegourd_halwa.md)
[Recipe 003 protein](./docs/Recipes/recipe_003_protein_fudge.md)
[Recipe 004 banana](./docs/Recipes/recipe_004_banana

# Parsing for cooklang notation

In [2]:
def parse_quantity(item: str) -> list[str, str]:
    """Parse the quantity portion of an ingredient
    e.g. 2%kg
    """
    if "%" not in item:
        return [item, ""]
    return item.split("%", maxsplit=1)


def parse_ingredient(item: str) -> dict[str, str]:
    """Parse an ingredient string
    eg. @salt or @milk{4%cup}
    """
    if item[0] != "@":
        raise ValueError("Ingredients should start with @")
    if item[-1] != "}":
        return {
            "type": "ingredient",
            "name": item[1:],
            "quantity": "some",
            "units": "",
        }
    name, quantity = item.split("{", maxsplit=1)
    val, units = parse_quantity(quantity[0:-1])
    return {
        "type": "ingredient",
        "name": name[1:],
        "quantity": val or "as needed",
        "units": units,
    }
def find_ingredients(step: str) -> list[str]:
    """Find ingredients in a recipe step"""
    return find_specials(step, "@")

def find_specials(step: str, start_char="#") -> list[str]:
    matches = []
    item = ""
    matching: bool = False
    specials = ["~", "@", "#"]
    for i, x in enumerate(step):
        if x == start_char:
            if start_char == "~" and step[i - 1] == "{":
                continue  # Skip - approx value in ingredient
            matching = True
            item += x
            continue
        if matching and x in specials:
            if " " in item:
                item = item.split(" ")[0]
            elif "." in item:
                item = item.split(".")[0]
            matches.append(item)
            matching = False
            item = ""
        if matching and x == "}":
            item += x
            matches.append(item)
            matching = False
            item = ""
        if matching:
            item += x

    if matching:
        if " " in item:
            item = item.split(" ")[0]
        elif "." in item:
            item = item.split(".")[0]
        matches.append(item)
    return matches

def obtain_ingredients(recipe):
    matches = []
    for item in find_ingredients(recipe):
        matches.append(parse_ingredient(item))
    ingredients = {}
    
    #matches
    for match in matches:
        ingredient_name = match['name'].title()
        amount = match['quantity']
        unit = match['units']

        ingredient_key = ingredient_name
        if ingredient_key in ingredients:
            ingredients[ingredient_key].append((amount, unit))
        else:
            ingredients[ingredient_key] = [(amount, unit)]
            
    return ingredients

# Include Filename with ingredients to identify which recipe it belongs to

In [3]:
input_string = ''
file_ingredients = {}
for result in results:
    filename = result.split(":FileName")[1]
    #input_string += result +"\n\n"
    ingredients = obtain_ingredients(result)
    file_ingredients[filename] = ingredients
file_ingredients.keys()
#ingredients

dict_keys([' recipe_1_tava_roti.md', ' recipe_2_wheat_gluten_bread.md', ' recipe_3_tandoori_roti.md', ' recipe_4_naan_bread.md', ' recipe_001_flaxseed_porridge.md', ' recipe_002_almond_bread.md', ' recipe_003_keto_dhokla.md', ' recipe_004_scrambled_tofu.md', ' recipe_005_keto_upma.md', ' recipe_001_peanut_mugcake.md', ' recipe_002_bottlegourd_halwa.md', ' recipe_003_protein_fudge.md', ' recipe_004_banana_bread.md', ' recipe_005_chocolate_cookies.md', ' test.md', ' recipe_007_mint_spinach_dip.md', ' recipe_008_green_pesto.md', ' recipe_009_tzatziki_dip.md', ' recipe_012_basil_mint_dip.md', ' recipe_001_sparkling_cinnamon.md', ' recipe_002_spicy_milk.md', ' recipe_001_keto_gatte.md', ' recipe_002_baigan_bharta.md', ' recipe_003_fried_okra.md', ' recipe_004_fried_cauliflower.md', ' recipe_005_soya_malai.md', ' recipe_006_low_carb_pizza.md', ' recipe_007_palak_mushroom.md', ' recipe_008_methi_malai_paneer.md', ' recipe_009_spicy_lauki.md', ' recipe_010_chutney_paneer.md', ' recipe_011_lahs

In [4]:
import pandas as pd
from fractions import Fraction

# Create an empty list to store data frames
dfs = []

# Iterate over the` dictionary items and create a data frame for each ingredient
for filename, ingredients in file_ingredients.items():
    ingredients = file_ingredients[filename]
    for ingredient, amounts in ingredients.items():
        temp_df = pd.DataFrame(amounts, columns=['Amount', 'Unit'])
        temp_df['Unit'] = temp_df['Unit'].str.upper()
        temp_df['Recipe_name'] = filename.upper()
        temp_df['Ingredient'] = ingredient.upper()
        dfs.append(temp_df)
# Concatenate the data frames into a single data frame
df = pd.concat(dfs, ignore_index=True)

# Print the DataFrame
df[['Recipe_name', 'Ingredient','Amount', 'Unit']]

,Recipe_name,Ingredient,Amount,Unit
0,RECIPE_1_TAVA_ROTI.MD,SOYA FLOUR,60,GMS(4 TABLESPOONS)
1,RECIPE_1_TAVA_ROTI.MD,WHEAT FLOUR,10,GMS(2 TEASPOONS)
2,RECIPE_1_TAVA_ROTI.MD,WARM WATER,4-6,TABLESPOONS
3,RECIPE_1_TAVA_ROTI.MD,GHEE,1/2,TEASPOON
4,RECIPE_2_WHEAT_GLUTEN_BREAD.MD,OLIVE OIL,1,TBSP
...,...,...,...,...
526,RECIPE_011_TOMATO_KEFTEDES.MD,VITAL WHEAT GLUTEN,if needed,
527,RECIPE_011_TOMATO_KEFTEDES.MD,ALMOND FLOUR,1/4,CUP
528,RECIPE_011_TOMATO_KEFTEDES.MD,BAKING POWDER,1,TSP
529,RECIPE_011_TOMATO_KEFTEDES.MD,EXTRA VIRGIN OLIVE OIL,1/2,TSP


# Update the all ingredients database to have uniform ingredient qty

In [5]:
import re

def replace_amount(value):
    # Replace fractions with decimals
    if '/' in value:
        numerator, denominator = value.split('/')
        try:
            value = str(float(numerator) / float(denominator))
        except ZeroDivisionError:
            value = '0'
    
    # Replace ranges with the highest value
    if '-' in value:
        value = value.split('-')[-1]
    
    # Replace worded items with 0
    if value.isalpha() or re.match(r'^[a-zA-Z\s]+$', value):
        value = '0'
    
    # Additional replacements
    value = value.lower().strip()
    if value == 'as needed' or value == 'to taste' or value == 'to taste (optional)':
        value = '0'
    elif value.endswith('l') or value.endswith('ml'):
        value = value[:-1]
    
    return value

# Apply the replacement function to the 'Amount' column
df['New Amount'] = df['Amount'].apply(replace_amount)
df['New Amount'].unique()

array(['60', '10', '6', '0.5', '1', '300', '0.25', '7', '5', '50', '250',
       '1.5', '2.5', '0.75', '0', '2', '3', '150', '30', '15', '100',
       '200', '8', '125', '120', '80', '4', '500', '750', '25', '40',
       '240', '24', '18', '20', '75', '35', '400', '16', '1.25', '90',
       '160'], dtype=object)

In [6]:
df['Unit'].unique()

array(['GMS(4 TABLESPOONS)', 'GMS(2 TEASPOONS)', 'TABLESPOONS',
       'TEASPOON', 'TBSP', 'TSP', 'ML', 'CUP', 'GMS(1 SACHET)', 'GMS',
       'TABLESPOON', 'HANDFUL', 'CUP(86G)', '', 'TABLESPOON (LUKEWARM)',
       'SPRAYS', 'GMS(MEDIUM SIZED)', 'GMS(MEDIUM SIZED CHOPPED)',
       'CLOVES(GRATED)', 'GMS(1-2 CHOPPED)', 'GMS(2 MEDIUM CHOPPED)',
       'TBSP(1 FRESHLY SQUEEZED)', 'GMS(1 BUNCH)', 'GMS(3 MEDIUM)',
       'GMS(2 CHOPPED)', 'CLOVE', 'CUP (HOT)', 'TEASPOONS', 'MEDIUM SIZE',
       'GMS(1 LARGE)', 'TBSP(1 INCH)', 'CLOVES', 'GMS (8-10)',
       'INCH (CHOPPED)', 'GMS (3 FINELY CHOPPED)', 'GMS( 1 MEDIUM)',
       'TSP (DEGI)', 'CUPS', 'GMS (2 MEDIUM CHOPPED)', 'TBSP (CHOPPED)',
       'GMS(1 FINELY CHOPPED)', 'GRATED', 'GMS(5 PIECES)',
       'GMS(10 PIECES)', 'TSP(5 PEELED CRUSHED)', 'GMS(3, VERY RIPE)',
       'L', 'GMS (GRATED)', 'LITRE', 'GMS(1/2 OF MIXER JAR)',
       'GMS(2-3 MEDIUM)', 'GMS(7-10 LEAVES)', 'INCH', 'GMS(4 MEDIUM)',
       'GMS(1/6 CHOPPED)', 'GMS(1 CHOPPED)',

# Likewise for unit

In [7]:
import re

def replace_unit(unit):
    # Remove information in brackets
    unit = re.sub(r'\(.*?\)', '', unit)
    
    # Remove trailing whitespace
    unit = unit.upper().strip()
    
    return unit

# Apply the replacement function to the 'Unit' column
df['New Unit'] = df['Unit'].apply(replace_unit)

# Check new dataframe with all ingredients and new coumns

In [8]:
df['New Unit'].unique()

array(['GMS', 'TABLESPOONS', 'TEASPOON', 'TBSP', 'TSP', 'ML', 'CUP',
       'TABLESPOON', 'HANDFUL', '', 'SPRAYS', 'CLOVES', 'CLOVE',
       'TEASPOONS', 'MEDIUM SIZE', 'INCH', 'CUPS', 'GRATED', 'L', 'LITRE',
       'MEDIUM', 'KG', 'TBSP CHOPPED', 'GM{1 PIECE', 'LEAF', 'BUNCH'],
      dtype=object)

In [9]:
df['Ingredient'].unique()

array(['SOYA FLOUR', 'WHEAT FLOUR', 'WARM WATER', 'GHEE', 'OLIVE OIL',
       'SUGAR', 'WATER', 'VITAL WHEAT GLUTEN', 'WHEAT BRAN', 'OAT FLOUR',
       'FLAX MEAL', 'ALMOND FLOUR', 'BAKING SODA', 'SALT', 'DRIED YEAST',
       'UNSALTED BUTTER', 'GREEK YOGURT (SOUR IS BETTER)', 'ONION SEEDS',
       'ALMONDS', 'WALNUTS', 'CASHEWS', 'PISTACHIOS', 'MACADEMIA NUTS',
       'GOLDEN FLAXSEED POWDER', 'FULL FAT GREEK YOGURT', 'ERYTHRITOL',
       'BLUEBERRIES', 'RASPBERIIES', 'BLANCHED ALMOND FLOUR',
       'BAKING POWDER', 'BAKING SPRAY', 'AVOCADO', 'ONION', 'GARLIC',
       'TOMATO', 'GREEN CHILLIES', 'LEMON', 'BLACK PEPPER',
       'FRESH CORRIANDER LEAVES', 'FRESH MINT', 'GREEN CHILLI',
       'BLACK MUSTARD', 'SUGARFREE TOMATO KETCHUP', 'GINGER', 'RED ONION',
       'AVOCADO OIL', 'CUMIN', 'TURMERIC', 'RED CHILLI POWDER', 'TOFU',
       'PINK HIMALAYAN SALT', 'PEPPER', 'PEANUTS', 'CASHEW NUTS',
       'EXTRA VIRGIN OLIVE OIL', 'MUSTARD SEEDS', 'CUMIN SEEDS',
       'CHANA DAL', 'URAD DAL

In [10]:
df

,Amount,Unit,Recipe_name,Ingredient,New Amount,New Unit
0,60,GMS(4 TABLESPOONS),RECIPE_1_TAVA_ROTI.MD,SOYA FLOUR,60,GMS
1,10,GMS(2 TEASPOONS),RECIPE_1_TAVA_ROTI.MD,WHEAT FLOUR,10,GMS
2,4-6,TABLESPOONS,RECIPE_1_TAVA_ROTI.MD,WARM WATER,6,TABLESPOONS
3,1/2,TEASPOON,RECIPE_1_TAVA_ROTI.MD,GHEE,0.5,TEASPOON
4,1,TBSP,RECIPE_2_WHEAT_GLUTEN_BREAD.MD,OLIVE OIL,1,TBSP
...,...,...,...,...,...,...
526,if needed,,RECIPE_011_TOMATO_KEFTEDES.MD,VITAL WHEAT GLUTEN,0,
527,1/4,CUP,RECIPE_011_TOMATO_KEFTEDES.MD,ALMOND FLOUR,0.25,CUP
528,1,TSP,RECIPE_011_TOMATO_KEFTEDES.MD,BAKING POWDER,1,TSP
529,1/2,TSP,RECIPE_011_TOMATO_KEFTEDES.MD,EXTRA VIRGIN OLIVE OIL,0.5,TSP


# Create all_recipe_ingredients.csv

In [11]:
df.to_csv('all_recipe_ingredients.csv')

# Create dataframes with unique unit and ingredients

In [12]:
unique_ing_df = df.drop_duplicates(subset='Ingredient')
unique_ing_df.to_csv('unique_ingredients.csv')

In [13]:
unique_unit_df = df.drop_duplicates(subset='New Unit')
unique_unit_df.to_csv('unique_units.csv')

# Functions used in parse_recipe

In [12]:
    import requests
    from thefuzz import fuzz, process

## read_upload

In [13]:
    def read_upload(obj_file,str_sheetname,col_list=None, date_col_list=False, file_type='xlsx', file_name=''):
        dataframe=""    
        try:
            if obj_file is not None:
                if file_type == 'csv':
                    dataframe = pd.read_csv(obj_file, usecols=col_list, parse_dates = date_col_list).applymap(lambda s: s.upper() if type(s) == str else s).fillna('')
                else:
                    dataframe = pd.read_excel(obj_file,sheet_name = str_sheetname, usecols=col_list, parse_dates = date_col_list).applymap(lambda s: s.upper() if type(s) == str else s).fillna('')
            else:
                dataframe= file_name+" is Empty - Provide a file name first"
                #print(dataframe)
        except ValueError as e:
            print("Problem: "+ e.args[0])
        finally:
            return dataframe

## get_indian_db_nutrient_information

In [14]:
    def get_indian_db_nutrient_information(ingredients):
        """ runs a fuzzy search on provided ingredient dataframe against indian_db"""
        #get data from csv file into a dataframe
        obj_file = 'indian_db.csv'
        str_sheetname = 'indian_db'
        col_list = ['Food Name; name', 'Energy; enerc', 'Total Fat; fatce', 'Dietary Fiber; fibtg', 'Carbohydrate; choavldf', 'Protein; protcnt']
        df_in_food_db = read_upload(obj_file,str_sheetname,col_list, date_col_list=False, file_type='csv', file_name='indian_db.csv')
        #rename columns
        df_in_food_db.rename(columns = { 'Food Name; name' : 'Food Name', \
                                        'Carbohydrate; choavldf' : 'Carbohydrate (g)', \
                                        'Protein; protcnt' : 'Protein (g)', \
                                        'Total Fat; fatce' : 'Fat (g)', \
                                        'Dietary Fiber; fibtg' : 'Fibre',
                                        'Energy; enerc' : 'Energy'}, inplace=True)
        #print(df_in_food_db.head(5))
        df_in_options_inner = pd.DataFrame()
        choices = df_in_food_db['Food Name'].unique()
        for item in ingredients:
            ingredient = item.upper()
            comp_var = process.extract(ingredient, choices, limit=6)
            print(comp_var)
            #print(item)
            if comp_var:
                for i in range(len(comp_var)):
                    if comp_var[i][1] >=90:
                        df_filtered = df_in_food_db[df_in_food_db['Food Name'] == comp_var[i][0]][['Food Name','Carbohydrate (g)', 'Protein (g)','Fat (g)','Energy','Fibre']]
                        df_filtered['Searched Ingredient'] = ingredient
                        df_in_options_inner = pd.concat([df_in_options_inner, df_filtered])
                        # if match was 100% then break from this for loop to get next item from outer for loop.
                        if comp_var[i][1] == 100:
                            break
        
        return df_in_options_inner

## get_nccdb_nutrient_information

In [15]:
        
    def get_nccdb_nutrient_information(api_key, ingredients, nutrient_ids=[1008, 1005, 1079, 1003, 1004]):
        """ runs a fuzzy search on provided ingredient list against ukdb"""
        #print(f'get_nccdb_nutrient_information --- {ingredients}')
        url = "https://api.nal.usda.gov/fdc/v1/foods/search"
        headers = {"Content-Type": "application/json"}
        nutrient_data = []  # List to store the extracted nutrient information
        df_options_nccdb_inner = pd.DataFrame()
        for ingredient in ingredients:
            #print(ingredient)
            params = {
                "query": ingredient,
                "pageSize": 10,
                "api_key": api_key
            }

            response = requests.get(url, headers=headers, params=params)
            response_json = response.json()

            foods = response_json.get('foods', [])
            #print(len(foods))
            if foods:
                for i in range(len(foods)):
                    food = foods[i]
                    #print(food['description'])
                    nutrient_info = {
                        'Food Name': ''.join(food['description'].upper().split(','))
                    }
                    for nutrient in food.get('foodNutrients', []):
                        if nutrient['nutrientId'] in nutrient_ids:
                            nutrient_info[nutrient['nutrientName']] = nutrient['value']
                    nutrient_data.append(nutrient_info)
                df_nccdb = pd.DataFrame(nutrient_data)
                patterns = ['RECIPE', 'CANS', 'BISCUIT', 'NO FRUIT PIECES', \
            'FLAVOURS','BEEF','DISH','LAMB','PIE','DESSERT','INCLUDING',\
            'SAMPLES','MANUFACTURER','CURRY','BRAMLEY','CLOVER','LIGHT MEAT','COOKED',\
           'LEAN AND','EDIBLE PORTION OF','EDIBLE CONVERSION FACTOR','BOILED','STEAMED',\
           'JARS','FISH','ROAST','FRIED','GRILLED', 'MEAT AND','JUICE DRINK', 'CARBONATED',\
           'KEBAB','CRAB','WEIGHT LOSS','FILLETS','CAKE','SALTED', 'SMOKED','GOOSE','HARE',\
            'HAM','BAKED','TURKEY','PUNJABI','TONGUE','SABJI','OX','TUNA','HOMEMADE','WITH SUGAR',\
            'STEWED','FAT FREE','PUDDING','SAUSAGE','PORK','PASTRY','RETAIL','SOUP','SQUID','STEAK',\
            'STEW','WEDGES','SWEETS','BOILED','JAM','INFUSION','BURGER','TOFFEE','TRIFLE','TOPPING',\
           'TURKEY','CASSEROLE','DIP','PICKLED','SAUCE']
                pattern = '|'.join(patterns)  # Combine multiple patterns with the OR operator

                df_nccdb = df_nccdb[~(df_nccdb['Food Name'].str.contains(pattern, flags=re.IGNORECASE))]
                choices = df_nccdb['Food Name'].unique()
                comp_var = process.extract(ingredient, choices, limit=6)
             #   print(comp_var)
              #  print(len(comp_var))

                if comp_var:
                    for i in range(len(comp_var)):
                        if comp_var[i][1] >=90:
                            df_filtered = df_nccdb[df_nccdb['Food Name'] == comp_var[i][0]].reset_index(drop=True)
                            df_filtered['Searched Ingredient'] = ingredient
                            df_options_nccdb_inner = pd.concat([df_options_nccdb_inner, df_filtered])
                            if comp_var[i][1] == 100:
                                break
                    #df_options_nccdb_inner['Searched Ingredient'] = ingredient

        #df = pd.DataFrame(nutrient_data)
        #print(df_options_nccdb_inner.keys())
        if not df_options_nccdb_inner.empty:
            df_options_nccdb_inner['Net Carbs'] = df_options_nccdb_inner['Carbohydrate, by difference'] - df_options_nccdb_inner['Fiber, total dietary'].fillna(0)
        #df_options_nccdb_inner.rename(columns = { 'Carbohydrate, by difference' : 'Carbohydrate (g)','Protein' : 'Protein (g)', 'Total lipid (fat)' : 'Fat (g)', 'Energy' : 'Energy (kcal) (kcal)'}, inplace=True)
        #Total lipid (fat) |   Carbohydrate, by difference |   Energy |   Fiber, total dietary
        #df = df_options_nccdb_inner[['Searched Ingredient','Food Name', 'Carbohydrate (g)', 'Fiber, total dietary', 'Net Carbs', 'Protein (g)', 'Fat (g)', 'Energy (kcal) (kcal)']]
        #print(df)
        return df_options_nccdb_inner#[['Searched Ingredient','Food Name', 'Carbohydrate (g)', 'Fiber, total dietary', 'Net Carbs', 'Protein (g)', 'Fat (g)', 'Energy (kcal) (kcal)']].drop_duplicates(subset='Food Name')     
    

## get_matches

In [16]:
    def get_matches(df):
        """ runs a fuzzy search on provided dataframe against ukdb"""
        #from rapidfuzz import process
        obj_file = './docs/assets/tables/McCance_Widdowsons_Composition_of_Foods_Integrated_Dataset_2021.xlsx'
        str_sheetname = '1.3 Proximates'
        col_list = ['Food Name','Description','Carbohydrate (g)', 'Protein (g)','Fat (g)','Energy (kcal) (kcal)','AOAC fibre (g)']
        df_uk_food_db = read_upload(obj_file,str_sheetname,col_list, date_col_list=False, file_type='xlsx', file_name='McCance_Widdowsons_Composition_of_Foods_Integrated_Dataset_2021.xlsx')
        #print(df_uk_food_db.shape)
        #df_uk_food_db = df_uk_food_db[~[df_uk_food_db['Description'] contains ['Recipe']]]
        patterns = ['RECIPE', 'CANS', 'BISCUIT', 'NO FRUIT PIECES', \
            'FLAVOURS','BEEF','DISH','LAMB','PIE','DESSERT','INCLUDING',\
            'SAMPLES','MANUFACTURER','CURRY','BRAMLEY','CLOVER','LIGHT MEAT','COOKED',\
           'LEAN AND','EDIBLE PORTION OF','EDIBLE CONVERSION FACTOR','BOILED','STEAMED',\
           'JARS','FISH','ROAST','FRIED','GRILLED', 'MEAT AND','JUICE DRINK', 'CARBONATED',\
           'KEBAB','CRAB','WEIGHT LOSS','FILLETS','CAKE','SALTED', 'SMOKED','GOOSE','HARE',\
            'HAM','BAKED','TURKEY','PUNJABI','TONGUE','SABJI','OX','TUNA','HOMEMADE','WITH SUGAR',\
            'STEWED','FAT FREE','PUDDING','SAUSAGE','PORK','PASTRY','RETAIL','SOUP','SQUID','STEAK',\
            'STEW','WEDGES','SWEETS','BOILED','JAM','INFUSION','BURGER','TOFFEE','TRIFLE','TOPPING',\
           'TURKEY','CASSEROLE','DIP','PICKLED','SAUCE']
        pattern = '|'.join(patterns)  # Combine multiple patterns with the OR operator

        df_uk_food_db = df_uk_food_db[~((df_uk_food_db['Description'].str.contains(pattern, flags=re.IGNORECASE))\
                                        | (df_uk_food_db['Food Name'].str.contains(pattern, flags=re.IGNORECASE)))]
        df_options = pd.DataFrame()
        df_options_nccdb = pd.DataFrame()
        df_options_indian_db = pd.DataFrame()
        filtered_list = []
        choices = df_uk_food_db['Food Name']
        print(df['Ingredient'].unique())
        for item in df['Ingredient'].unique():
            comp_var = process.extract(item, choices, limit=6)
            #print(comp_var)
            #print(item)
            if comp_var:
                for i in range(len(comp_var)):
                    if comp_var[i][1] >=90:
                        df_filtered = df_uk_food_db[df_uk_food_db['Food Name'] == comp_var[i][0]][['Food Name','Carbohydrate (g)', 'Protein (g)','Fat (g)','Energy (kcal) (kcal)','AOAC fibre (g)']]
                        df_filtered['Searched Ingredient'] = item
                        df_options = pd.concat([df_options, df_filtered])
                        if comp_var[i][1] == 100:
                            break
        possible_matches = ''
        api_key = 'DEMO_KEY'
        
        if df_options.empty:
            possible_matches = ''
            ### Pass the whole ingredient list to nccdb as nothing was found on ukdb
            filtered_list = df['Ingredient'].to_list()
            #print(filtered_list)
        else:
            df_options = df_options[['Searched Ingredient','Food Name', 'Carbohydrate (g)', 'Protein (g)', 'Fat (g)','Energy (kcal) (kcal)', 'AOAC fibre (g)']]
            possible_matches_res = '\n\t\t' + df_options.to_markdown(index=False).replace('\n','\n\t\t')
            possible_matches = f'\t\t*Possible matches in [McCance and Widdowson\'s composition of foods integrated dataset](https://www.gov.uk/government'+\
                f'/publications/composition-of-foods-integrated-dataset-cofid#full-publication-update-history)'+\
                "{target=_blank}"+ f' are shown below:*\n{possible_matches_res}'
            ### Pass the ingredients not found on ukdb to indian_db
            filtered_list = df['Ingredient'][~(df['Ingredient'].isin(df_options['Searched Ingredient']))].to_list()

        ### Call indian_db for ingredient not available on ukdb
        if filtered_list:              
            df_options_indian_db = get_indian_db_nutrient_information(filtered_list).fillna(0)
            print(f'filtered_list before calling indian db: {filtered_list}')
        
        if df_options_indian_db.empty:
            possible_matches += ''
        else:            
            df_options_indian_db = df_options_indian_db.drop_duplicates(subset='Food Name')
            df_options_indian_db = df_options_indian_db[['Searched Ingredient', 'Food Name', 'Carbohydrate (g)', 'Protein (g)', 'Fat (g)', 'Energy', 'Fibre']]
            possible_matches_indian_db = '\n\n\t\t' + df_options_indian_db.to_markdown(index=False).replace('\n','\n\t\t')
            possible_matches += f'\n\n\t\t*Possible matches in [Indian Food Composition Database](https://ifct2017.com/frame.php?page=food'+\
            f')'+\
            "{target=_blank}"+ f' are shown below:*\n{possible_matches_indian_db}'
            
            ## Reduce filtered list to items not found in indian_db
            filtered_list = [item for item in filtered_list if item.upper() not in df_options_indian_db['Searched Ingredient'].values]
            #print(f'filtered_list before calling nccdb: {filtered_list}')
            #print(df_options_indian_db['Searched Ingredient'])
            
        ### Call nccdb for ingredients not available on indian_db
        if filtered_list:
            try:
                df_options_nccdb = get_nccdb_nutrient_information(api_key, filtered_list).fillna(0)
            except:
                pass
        if df_options_nccdb.empty:
            possible_matches += ''
        else:
            df_options_nccdb.rename(columns = { 'Carbohydrate, by difference' : 'Carbohydrate (g)','Protein' : 'Protein (g)', 'Total lipid (fat)' : 'Fat (g)', 'Energy' : 'Energy (kcal) (kcal)'}, inplace=True)
            df_options_nccdb = df_options_nccdb[['Searched Ingredient','Food Name', 'Carbohydrate (g)', 'Fiber, total dietary', 'Net Carbs', 'Protein (g)', 'Fat (g)', 'Energy (kcal) (kcal)']].drop_duplicates(subset='Food Name')
            possible_matches_nccdb = '\n\n\t\t' + df_options_nccdb.to_markdown(index=False).replace('\n','\n\t\t')
            possible_matches += f'\n\n\t\t*Possible matches in [U.S. Department of Food Central database](https://fdc.nal.usda.gov/fdc-app.html#/'+\
            f')'+\
            "{target=_blank}"+ f' are shown below:*\n{possible_matches_nccdb}'

        
        #print(possible_matches)
        return possible_matches
        #return df_options_nccdb,df_options

## get_unformatted_line

In [17]:
    
    def get_unformatted_line(input_string: str):
        """Takes input_string as input, and removes all cookland notation from steps 
        and retains metadata as is and returned the steps as a list of lines"""

        #Remove individual timer notation ~{25%minutes} or ~{25-30%minutes}
        pattern = r'~{(\d+)(?:-(\d+))?%([^}]+)}'
        replacement = lambda match: f"{match.group(1)}-{match.group(2)} {match.group(3)}" if match.group(2) else f"{match.group(1)} {match.group(3)}"
        input_string = re.sub(pattern, replacement,input_string)
        #split resulting steps with time formatting removed further remove ingredient and cookware formatting
        lines = input_string.replace("{}", '').\
            replace("@", "").\
            replace("%", " ").\
            replace("#", '').\
            replace("~", '').\
            replace("{", ' (').\
            replace("}", ')').\
            splitlines()
        return lines

## puml

In [29]:
    def puml(input_string: str):
        inp_str = ''
        style_str = """
            <style>
            activityDiagram {
              diamond {
                BackgroundColor #darkgreen
                LineColor #white
                FontColor white
              }
            }
            </style>
        """
        lines = get_unformatted_line(input_string)
        #Remove metadata of cooklang that starts with >> and store it in variable inp_str
        for line in lines:
            if line.strip() != "" and not line.startswith(">>"):
                inp_str += f'{line.strip()}\n'
        #steps = inp_str.split('\n')
        steps = inp_str.splitlines()
        steps_string = "<div class=\"grid cards\" markdown>\n\n\n-   ## Steps\n\n\t---"
        out = "\n-   ## Process\n\n\t---\n\n\t```plantuml\n\t@startuml\n\t!theme aws-orange\n\t"+style_str+"\n\tstart\n"
        for step in steps:
            # Convert step into uppercase for uniform comparison
            p_step = step.replace("`","").upper()
            # Check if step starts with IF and contains a THEN 
            # If so, it is a candidate for If Then Else syntax of plantuml
            # If not treat it as normal step
            if p_step.startswith('IF') and 'THEN' in p_step:
                # Replace 'ELSE IF' with 'ELSEIF' so there is no clash with final ELSE statement
                if 'ELSE IF' in p_step:
                    p_step = p_step.replace('ELSE IF','ELSEIF')
                # Create a variable to first remove just IF from the step
                if_removed = ''.join(re.split(r"\bIF\b", p_step)).strip()
                # Using above, remove 'ELSE'. This will be a list with two items
                else_removed_l = re.split(r"\bELSE\b", if_removed)
                # Now will If and Else removed, break the sentencefirst item from above list
                # at ELSEIF and store in another list below
                elif_removed_l = re.split(r"\bELSEIF\b", else_removed_l[0])
                # For every item in above list, break it down at THEN and store in a new list
                then_removed_l = []
                for elif_removed in elif_removed_l:
                    then_removed_l += re.split(r"\bTHEN\b", elif_removed)
                # Initiate if loop parsing
                i = 0
                while i < len(then_removed_l): 
                    if i==0:
                        # The very first entry in then_removed_l is condition for if statement 
                        # and second entry is then statement
                        out += f'\tif ({insert_newlines(then_removed_l[i].strip(),20)}?) then (yes)\n\t\t:{insert_newlines(then_removed_l[i+1].strip().capitalize(),30)};\n'
                        i = i+2
                    elif i % 2 == 0:
                        # Add elseif condition 
                        #Logic is that variable then_removed_l has every even item as an elseif condition 
                        # and every odd item as then statement
                        out+= f'\t(no) elseif ({insert_newlines(then_removed_l[i].strip(),20)}?) then (yes)\n'
                        i = i+1
                    else:
                        #Every odd entry is a then statement so use it to create the then statement
                        out+=f'\t\t:{insert_newlines(then_removed_l[i].strip().capitalize(),30)};\n'
                        i = i + 1
                # Check if ELSE exists in the step and if it does include the final else statement
                if len(else_removed_l)>1:
                    out += f'\telse (no)\n\t\t:{insert_newlines(else_removed_l[1].strip().capitalize(),30)};\n'
                else:
                    out += f'\telse (no)\n\t\t\n'
                out += f'\tendif\n'
            # Ignore empty line in steps
            elif step != '':
                if step.startswith('**') and step.endswith('**'):
                    # If the step starts with ** and ends with **, apply different formatting and remove **
                    step = step.replace("**","")
                    out += f'\t#Black:**{insert_newlines(step.replace("`","").strip(),50)}**/\n'
                    step_line = f"\n\n\t### {step}\n\n"
                else:
                    # If the step does not start with ** and ends with **, apply standard formatting
                    out += f'\t:{insert_newlines(step.replace("`","").strip(),50)};\n'
                    step_line = f"\n\t* {step.strip()}"
                steps_string += step_line
        out += f'\tend\n\t@enduml\n\t```\n\n</div>\n\n'
        out = f'{steps_string}\n\n{out}'
        # Return final markdown for steps and plantuml
        return out

In [30]:
print(puml("If it's too spicy then add a little bit of @erythritol{as needed}."))

<div class="grid cards" markdown>


-   ## Steps

	---


-   ## Process

	---

	```plantuml
	@startuml
	!theme aws-orange
	
        <style>
        activityDiagram {
          diamond {
            BackgroundColor #darkgreen
            LineColor #white
            FontColor white
          }
        }
        </style>
    
	start
	if (IT'S TOO SPICY?) then (yes)
		:Add a little bit of erythritol 
	(as needed).;
	else (no)
		
	endif
	end
	@enduml
	```

</div>




## Core Cooklang Parsing functions

In [19]:
    def parse_cookware(item: str) -> dict[str, str]:
        """Parse cookware item
        e.g. #pot or #potato masher{}
        """
        if item[0] != "#":
            raise ValueError("Cookware should start with #")
        item = item.replace("{}", "")
        return item[1:]


    def parse_quantity(item: str) -> list[str, str]:
        """Parse the quantity portion of an ingredient
        e.g. 2%kg
        """
        if "%" not in item:
            return [item, ""]
        return item.split("%", maxsplit=1)


    def parse_ingredient(item: str) -> dict[str, str]:
        """Parse an ingredient string
        eg. @salt or @milk{4%cup}
        """
        if item[0] != "@":
            raise ValueError("Ingredients should start with @")
        if item[-1] != "}":
            return {
                "type": "ingredient",
                "name": item[1:],
                "quantity": "some",
                "units": "",
            }
        name, quantity = item.split("{", maxsplit=1)
        val, units = parse_quantity(quantity[0:-1])
        return {
            "type": "ingredient",
            "name": name[1:],
            "quantity": val or "as needed",
            "units": units,
        }


    def parse_timer(item: str) -> dict[str, str]:
        """Parse timer string
        e.g. ~eggs{3%minutes} or ~{25%minutes}
        """
        if item[0] != "~":
            raise ValueError("Timer should start with ~")
        name, quantity = item.split("{", maxsplit=1)
        val, units = parse_quantity(quantity[0:-1])
        return {
            "type": "timer",
            "name": name[1:],
            "quantity": val,
            "units": units,
        }

    def find_specials(step: str, start_char="#") -> list[str]:
        matches = []
        item = ""
        matching: bool = False
        specials = ["~", "@", "#"]
        for i, x in enumerate(step):
            if x == start_char:
                if start_char == "~" and step[i - 1] == "{":
                    continue  # Skip - approx value in ingredient
                matching = True
                item += x
                continue
            if matching and x in specials:
                if " " in item:
                    item = item.split(" ")[0]
                elif "." in item:
                    item = item.split(".")[0]
                matches.append(item)
                matching = False
                item = ""
            if matching and x == "}":
                item += x
                matches.append(item)
                matching = False
                item = ""
            if matching:
                item += x

        if matching:
            if " " in item:
                item = item.split(" ")[0]
            elif "." in item:
                item = item.split(".")[0]
            matches.append(item)
        return matches

    def find_cookware(step: str) -> list[str]:
        """Find cookware items in a recipe step"""
        return find_specials(step, "#")


    def find_ingredients(step: str) -> list[str]:
        """Find ingredients in a recipe step"""
        return find_specials(step, "@")


    def find_timers(step: str) -> list[str]:
        """Find timers in a recipe step"""
        return find_specials(step, "~")

    def insert_newlines(input_string: str, chars_per_line: int):
        """Inserts newline in the input_string after specified number of characters from char_per_line"""
        words = input_string.split()
        mod = ""
        count = 0
        for word in words:
            if count + len(word) > chars_per_line:
                mod += "\n\t"
                count = 0
            mod += word + " "
            count += len(word) + 1
        return mod.strip()

# Main parse_recipe function

In [20]:
    import re
    import os
    import pandas as pd
    from fractions import Fraction
    import requests
    from thefuzz import fuzz, process
    def parse_recipe(input_string):
        cooklang_block = f'\n??? abstract "Recipe in [Cooklang](https://cooklang.org/)' + '{target=_blank' + '}"\n\t```\n\t' + input_string.replace("\n","\n\t") + '```'
        ingredients = {}
        cookwares = set()
        steps = []
        cooking_data = {}
        ################### Extract Ingredients ###########################
        matches = []
        for item in find_ingredients(input_string):
            matches.append(parse_ingredient(item))
        for match in matches:
            ingredient_name = match['name'].title()
            amount = match['quantity']
            unit = match['units']

            ingredient_key = ingredient_name
            if ingredient_key in ingredients:
                ingredients[ingredient_key].append((amount, unit))
            else:
                ingredients[ingredient_key] = [(amount, unit)]
                
        ##################### Extract cookwares ############################
        cookware_matches = find_cookware(input_string)
        for cookware_match in cookware_matches:
            cookware_name = parse_cookware(cookware_match).title()
            cookwares.add(cookware_name)
        
        ##################### Extract cooking data and steps ################
        lines = get_unformatted_line(input_string)
        
        for line in lines:
            if line.strip() != "" and line.startswith(">>"):
                key, value = line.lstrip(">> ").strip().split(": ")
                cooking_data[key.strip()] = value.strip()
            elif line.strip() != "":
                steps.append(line.strip())
        ###################### Ingredient Block ########################
        ingredient_string = ""
        ingredient_string += "<div class=\"grid cards\" markdown>\n\n\n-   ## Ingredients\n\n\t---\n"
        #ingredient_string += "\n## Ingredients\n\n\t---\n"
        ingredient_count = 1
        for ingredient_name in ingredients.keys():
            #check if ingredient has been used more than once so it can be listed accordingly
            if len(ingredients[ingredient_name]) > 1:
                ingredient_line = f"\t\t{ingredient_count}. {ingredient_name}:"
                ingredient_string += ingredient_line + "\n"
                for amount, unit in ingredients[ingredient_name]:
                    ingredient_line = f"\t\t\t- {amount} {unit}"
                    ingredient_string += ingredient_line + "\n"
            else:
                # if ingredient is used only once, list as a single line with amount and units in same line
                ingredient_line = ""
                for amount, unit in ingredients[ingredient_name]:
                    ingredient_line += f"\t\t{ingredient_count}. {ingredient_name}: {amount} {unit}"
                ingredient_string += ingredient_line + "\n"
            ingredient_count += 1
        ###################### Ingredient Listing ########################    
        if cookwares:
            cookware_string = "\n-   ## Cookwares\n\n\t---\n"
            cookware_count = 1
            for cookware in cookwares:
                cookware_line = f"\t{cookware_count}. *{cookware}*"
                cookware_count += 1
                cookware_string += cookware_line + "\n"
        else:
            cookware_string = ''
        cookware_string += '\n\n</div>\n\n'
        if "Title" in cooking_data:
            title = cooking_data["Title"]
            del cooking_data["Title"]
            cooking_data_string = f"## {title}\n\n"
        else:
            cooking_data_string = ""
        ######################### NET CARB TABLE #################
        df_ingredient_db = pd.read_csv('ingredient_db.csv')
        df_unit_db = pd.read_csv('unit_db.csv')
        # Create an empty list to store data frames
        dfs = []
        # Iterate over the dictionary items and create a data frame for each ingredient
        for ingredient, amounts in ingredients.items():
            temp_df = pd.DataFrame(amounts, columns=['Amount', 'Unit'])
            temp_df['Ingredient'] = ingredient
            dfs.append(temp_df)
        # Concatenate the data frames into a single data frame
        df = pd.concat(dfs, ignore_index=True)
        # Now merge df and df_ingredient_db with inner join to get net carb values for recipe ingredients using code below:
        df_merge = df.merge(df_ingredient_db, how='inner', left_on=df['Ingredient'].str.upper(), right_on=df_ingredient_db['Name'].str.upper())
        # Define functions to calculate net carbs, conversion factor etc
        def replace_amount(value):
            # Replace fractions with decimals
            if '/' in value:
                numerator, denominator = value.split('/')
                try:
                    value = str(float(numerator) / float(denominator))
                except ZeroDivisionError:
                    value = '0'

            # Replace ranges with the highest value
            if '-' in value:
                value = value.split('-')[-1]

            # Replace worded items with 0
            if value.isalpha() or re.match(r'^[a-zA-Z\s]+$', value):
                value = '0'

            # Additional replacements
            value = value.lower().strip()
            if value == 'as needed' or value == 'to taste' or value == 'to taste (optional)':
                value = '0'
            elif value.endswith('l') or value.endswith('ml'):
                value = value[:-1]

            return value
        def replace_unit(unit):
            # Remove information in brackets
            unit = re.sub(r'\(.*?\)', '', unit)

            # Remove trailing whitespace
            unit = unit.upper().strip()

            return unit

        def conv_factor(row):
            filtered_units = df_unit_db[df_unit_db['Unit'].str.upper() == replace_unit(row['Unit_x'])]['eq_gms']
            if not filtered_units.empty:
                conv_factor = float(filtered_units.values[0])
                return conv_factor
            return 0
        def calculate_cal_net_carb(row):
            filtered_units = df_unit_db[df_unit_db['Unit'].str.upper() == replace_unit(row['Unit_x'])]['eq_gms']
            if not filtered_units.empty:
                gms_used_in_recipe = float(filtered_units.values[0]) * float(Fraction(replace_amount(row['Amount'])))
                if row['Net_carb/100gms'] != '0' and row['Net_carb/100gms'] != 0:
                    return (gms_used_in_recipe * float(row['Net_carb/100gms']))/100
            return 0
        def calculate_amt_in_gms(row):
            filtered_units = df_unit_db[df_unit_db['Unit'].str.upper() == replace_unit(row['Unit_x'])]['eq_gms']
            if not filtered_units.empty:
                gms_used_in_recipe = float(filtered_units.values[0]) * float(Fraction(replace_amount(row['Amount'])))
                return gms_used_in_recipe
            return 0
        # Add calculated columns to the dataframe:
        df_merge['Conversion Factor'] = df_merge.apply(conv_factor, axis=1)
        df_merge['Amount used in Recipe(gms)'] = df_merge.apply(calculate_amt_in_gms, axis=1)
        df_merge['Calculated Net Carb in recipe'] = df_merge.apply(calculate_cal_net_carb, axis=1).round(2)
        df_merge['Unit_x'] = df_merge['Unit_x'].astype(str).apply(replace_unit)
        df_merge.rename(columns = {'Unit_x' : 'Recipe Unit', 'Unit_y' : 'Converted Unit'}, inplace=True)
        
        # Create final markdown table:
        grand_total = f"**{df_merge['Calculated Net Carb in recipe'].sum().round(2)}**"
        result = df_merge[['Ingredient', 'Amount','Recipe Unit','Conversion Factor', 'Amount used in Recipe(gms)', 'Net_carb/100gms','Source','Calculated Net Carb in recipe']]

        # Create a DataFrame for the grand total
        grand_total_df = pd.DataFrame({'Source':'-','Net_carb/100gms': '-','Amount':'-','Recipe Unit': '-','Ingredient': ['**Grand Total**'], 'Calculated Net Carb in recipe': [grand_total], 'Conversion Factor': '-','Amount used in Recipe(gms)': '-' })

        # Concatenate the result DataFrame with the grand total DataFrame
        result = pd.concat([result, grand_total_df], ignore_index=True)
        
        # Find ingredients that are not on ingredient_db
        df_ingredients_not_found =  df[~df['Ingredient'].str.upper().isin(df_ingredient_db['Name'].str.upper())]
        not_found = ''
        if df_ingredients_not_found.empty:
            not_found = ''
        else:
            not_found = ', '.join(df_ingredients_not_found['Ingredient'].unique())
            possible_match = get_matches(df_ingredients_not_found)
            
        # Add the result DataFrame as markdown 
        netcarb_string = f'???+ Info "Calculated Net Carb Info (Total Net Carbs for entire dish: {grand_total})"\n\t' + result.to_markdown(index=False).replace("\n","\n\t")
        if not_found != '':
            netcarb_string += '\n\n\t!!! warning "Caution"\n\t\t*The calculation is indicative and my lookup list'+\
            ' did not have net carb values for* **' +\
            not_found +\
            '** *and thus not included in the calculations above.*\n\n'
            if possible_match != '': 
                netcarb_string += possible_match
        ######################### NET CARB TABLE ####################################
        
        ########################### COOKING DATA STRING #############################
        if "Image" in cooking_data:
            if "Image-Caption" in cooking_data:
                image_data_string = f"## Image\n\n<figure markdown>\n![image](../../assets/images/"+cooking_data["Image"]+"){: style=\"width: 920px;height: 430px;object-fit: contain;\"}\n<figcaption>" + cooking_data["Image-Caption"] + "</figcaption>\n</figure>\n\n"
                del cooking_data["Image-Caption"]
                del cooking_data["Image"]
            else:
                image_data_string = f"## Image\n\n<figure markdown>\n![image](../../assets/images/"+cooking_data["Image"]+"){: style=\"width: 920px;height: 430px;object-fit: contain;\"}\n</figure>\n\n"
                del cooking_data["Image"]
        else:
            image_data_string = ""        
        #cooking_data_string += "<div class=\"grid cards\" markdown>\n\n"
        temp_cooking_data_string = ""
        one_cooking_data_string = ""
        two_cooking_data_string = ""
        three_cooking_data_string = ""
        four_cooking_data_string = ""
        for key, value in cooking_data.items():
            if key in ('Cooking Time','Serving Size','Type', 'Source'):
                if key == 'Cooking Time':
                    one_cooking_data_string = f":material-timer: *{value}*"
                elif key == 'Serving Size':
                    two_cooking_data_string = f", :fontawesome-solid-chart-pie: *{value}*"
                    if value.split(' ')[0].isdigit():
                        net_carb_per_serving = round((float(grand_total.replace('*', '')) / int(value.split(' ')[0])), 2)
                        net_carb_per_serving_string = f', **Net Carbs per serving:** *[{net_carb_per_serving}](#nutritional-info)* '
                        #print(value.split(' '))
                    else:
                        net_carb_per_serving_string = ''
                elif key == 'Type':
                    if value == 'Vegetarian':
                        three_cooking_data_string = f", **{key}**: :leafy_green:"
                    elif value == 'Vegetarian with Egg':
                        three_cooking_data_string = f", **{key}**: :leafy_green::egg:"
                    else:
                        three_cooking_data_string = f", **{key}**: :cut_of_meat:"
                elif key == 'Source':
                    four_cooking_data_string = f", **{key}**: [:material-origin:]({value})"                
            else:
                cooking_data_string += f", **{key}**: *{value}*"
        if one_cooking_data_string != "":
            temp_cooking_data_string = one_cooking_data_string 
        if two_cooking_data_string != "":
            temp_cooking_data_string += two_cooking_data_string
        if three_cooking_data_string != "":
            temp_cooking_data_string += three_cooking_data_string
        if four_cooking_data_string != "":
            temp_cooking_data_string += four_cooking_data_string + "{target=_blank}"
        cooking_data_string = f'<div class=\"grid cards\" align = \"center\" markdown>\n\n-   ## Key Stats\n\n\t---\n\n\t   ' +\
        temp_cooking_data_string + cooking_data_string +\
        net_carb_per_serving_string + \
        f', **Total Net Carbs:** [{grand_total}](#nutritional-info) '+\
        '\n\n</div>\n\n'
        ############################## Cooking Data ################################
        steps_dia_string = puml(input_string)
        
        
        
        final_output_string = '\n\n' + image_data_string + cooking_data_string + "\n" + \
        ingredient_string + "\n" + cookware_string + "\n\n" +\
        steps_dia_string + '\n\n## Nutritional Info\n\n' + netcarb_string + "\n\n\n"+ cooklang_block 
        return final_output_string

# Check for recipes in dataframe to see if it came correctly

In [21]:
input_string="""
>> Serving Size: 16
>> Cooking Time: 60 minutes
>> Category: Greek
>> Type: Vegetarian
>> Image: recipe_008_low_carb_edamame_falafels.jpg
>> Image-Caption: Low Carb Edamame Falafel

Take @frozen edamame beans{250%gms} and @frozen peas{50%gms} in a #mixer-grinder jar{}.
Add @fresh mint leaves{30%gms}, @green chillies{10%gms (2-3)}, @garlic{2%cloves}, @fresh lemon juice{1/2%tbsp}.
Add @extra virgin olive oil{2%tbsp}, @cumin{1%tsp}, @Pink Himalayan Salt{to taste}.
Grind it on the #mixer-grinder{} till it becomes fine powder / paste texture.
Transfer to a #bowl{} and add @vital wheat gluten{1/4%cup}.
Now using hands mix it and then make 16 small flat and round shaped falafels while its still very cold.
If it's too spicy then add a little bit of @erythritol{as needed}.
Leave them to rest at room temperature for ~{20%minutes}.
Put them in #Air Fryer{} at 200C for ~{20%minutes}.
Serve hot.
"""
print(parse_recipe(input_string))



## Image

<figure markdown>
![image](../../assets/images/recipe_008_low_carb_edamame_falafels.jpg){: style="width: 920px;height: 430px;object-fit: contain;"}
<figcaption>Low Carb Edamame Falafel</figcaption>
</figure>

<div class="grid cards" align = "center" markdown>

-   ## Key Stats

	---

	   :material-timer: *60 minutes*, :fontawesome-solid-chart-pie: *16*, **Type**: :leafy_green:, **Category**: *Greek*, **Net Carbs per serving:** *[1.51](#nutritional-info)* , **Total Net Carbs:** [**24.17**](#nutritional-info) 

</div>


<div class="grid cards" markdown>


-   ## Ingredients

	---
		1. Frozen Edamame Beans: 250 gms
		2. Frozen Peas: 50 gms
		3. Fresh Mint Leaves: 30 gms
		4. Green Chillies: 10 gms (2-3)
		5. Garlic: 2 cloves
		6. Fresh Lemon Juice: 1/2 tbsp
		7. Extra Virgin Olive Oil: 2 tbsp
		8. Cumin: 1 tsp
		9. Pink Himalayan Salt: to taste 
		10. Vital Wheat Gluten: 1/4 cup
		11. Erythritol: as needed 


-   ## Cookwares

	---
	1. *Mixer-Grinder*
	2. *Mixer-Grinder Jar*
	

In [24]:
input_string = ''
print(len(results))
input_string = results[9].split(':FileName')
print(input_string[0])

42
>> Serving Size: 2 portions
>> Cooking Time: 3 minutes
>> Category: Dessert
>> Type: Vegetarian with Egg
>> Source: https://kutt.it/peanut_butter_mugcake
Add @Baking Powder{1/2%Tablespoon} and @Erythritol{1%Tablespoon} in a #small bowl{}
Add @Egg{1} and @Smooth Peanut Butter{3%Tablespoons} into the bowl.
Mix until well combined and smooth
Now coat a #small mug{} with @baking spray{}
Move the batter to the #small mug{}
#Microwave{} for 30 seconds to 1 minute




# Test whole macro function

In [25]:
print(parse_recipe(results[9]))



<div class="grid cards" align = "center" markdown>

-   :material-timer: *3 minutes*, :fontawesome-solid-chart-pie: *2 portions*, **Type**: :leafy_green::egg:, **Source**: [:material-origin:](https://kutt.it/peanut_butter_mugcake){target=_blank}, **Category**: *Dessert* **Total Net Carbs:** **7.47**

</div>


<div class="grid cards" markdown>


-   ## Ingredients

	---
		1. Baking Powder: 1/2 Tablespoon
		2. Erythritol: 1 Tablespoon
		3. Egg: 1 
		4. Smooth Peanut Butter: 3 Tablespoons
		5. Baking Spray: as needed 


-   ## Cookwares

	---
	1. *Microwave*
	2. *Small Mug*
	3. *Small Bowl*


</div>


??? Info "Calculated Net Carb Info (Total Net Carbs for entire dish: **7.47**)"
	| Ingredient           | Amount    | Recipe Unit   | Conversion Factor   | Amount used in Recipe(gms)   | Net_carb/100gms   | Calculated Net Carb in recipe   |
	|:---------------------|:----------|:--------------|:--------------------|:-----------------------------|:------------------|:------------------------

# Generate TOC Full max first

In [9]:
    import os

    def generate_toc_full(directory):
        def generate_toc_files(directory, indent='', include_dir_name_flag=False):
            output = ''
            # Generate TOC for files in the current directory
            files = [f for f in os.listdir(directory) if f.endswith('.md')]
            sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)))
            for file_name in sorted_files:
                recipe_name = ' '.join(file_name.split('.')[0].split('_')[2:]).title()
                if recipe_name != '':
                    if '\\' in directory:
                        dir_name_mod = directory.split('\\')[1].replace(' ', '%20')
                    else:
                        dir_name_mod_l = directory.split('/')
                        dir_name_mod = dir_name_mod_l[len(dir_name_mod_l) - 1].replace(' ', '%20')
                    if include_dir_name_flag:
                        output += f'{indent}1. [{recipe_name}](./{dir_name_mod}/{file_name})\n'  # Add the file to the TOC with indentation
                    else:
                        output += f'{indent}1. [{recipe_name}](./{file_name})\n'  # Add the file to the TOC with indentation

            return output

        def generate_toc_dirs(directory, indent=''):
            output = ''
            # Recursively generate TOC for subdirectories
            subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
            sorted_subdirs = sorted(subdirs, key=lambda x: len(os.listdir(os.path.join(directory, x))), reverse=True)
            for dir_name in sorted_subdirs:
                subdir = os.path.join(directory, dir_name)
                output += f'\n{indent}1. [**{dir_name}**](./{dir_name})\n\n    ---\n\n'  # Add the directory to the TOC with indentation
                output += generate_toc_files(subdir, indent + '    ', include_dir_name_flag=True)

            return output

        output = generate_toc_files(directory)
        output += generate_toc_dirs(directory)

        return output

print(generate_toc_full('./docs/Recipes'))


1. [**Main Meal Options**](./Main Meal Options)

    ---

    1. [Keto Gatte](./Main%20Meal%20Options/recipe_001_keto_gatte.md)
    1. [Baigan Bharta](./Main%20Meal%20Options/recipe_002_baigan_bharta.md)
    1. [Fried Okra](./Main%20Meal%20Options/recipe_003_fried_okra.md)
    1. [Fried Cauliflower](./Main%20Meal%20Options/recipe_004_fried_cauliflower.md)
    1. [Soya Malai](./Main%20Meal%20Options/recipe_005_soya_malai.md)
    1. [Low Carb Pizza](./Main%20Meal%20Options/recipe_006_low_carb_pizza.md)
    1. [Palak Mushroom](./Main%20Meal%20Options/recipe_007_palak_mushroom.md)
    1. [Methi Malai Paneer](./Main%20Meal%20Options/recipe_008_methi_malai_paneer.md)
    1. [Spicy Lauki](./Main%20Meal%20Options/recipe_009_spicy_lauki.md)
    1. [Chutney Paneer](./Main%20Meal%20Options/recipe_010_chutney_paneer.md)
    1. [Lahsuni Paneer](./Main%20Meal%20Options/recipe_011_lahsuni_paneer.md)

1. [**Snacks and Sides**](./Snacks and Sides)

    ---

    1. [Protein Cutlets](./Snacks%20and%20Si

# Generate TOC Full - old one - not in use now

In [46]:
import os

def generate_toc_full(directory):
    def generate_toc(directory, indent='',include_dir_name_flag = False):
        output = ''
        # Generate TOC for files in the current directory
        files = [f for f in os.listdir(directory) if f.endswith('.md')]
        sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)))
        for file_name in sorted_files:
            #if file_name not in ['tags.md','index.md','test.md']:
                recipe_name = ' '.join(file_name.split('.')[0].split('_')[2:]).title()
                if recipe_name != '':
                    if '\\' in directory:
                        dir_name_mod = directory.split('\\')[1].replace(' ','%20')
                    else:
                        dir_name_mod_l = directory.split('/')
                        dir_name_mod = dir_name_mod_l[len(dir_name_mod_l)-1].replace(' ','%20')
                    if include_dir_name_flag:
                        output += f'{indent}- [{recipe_name}](./{dir_name_mod}/{file_name})\n'  # Add the file to the TOC with indentation
                    else:
                        output += f'{indent}- [{recipe_name}](./{file_name})\n'  # Add the file to the TOC with indentation
             #   elif file_name == '00-prepare-to-beat.md':
             #       output += f'{indent}1. [Prepare for diabetic diet](./{file_name})\n'  # Add the file to the TOC with indentation

        # Recursively generate TOC for subdirectories
        subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
        sorted_subdirs = sorted(subdirs)
        for dir_name in sorted_subdirs:
            subdir = os.path.join(directory, dir_name)
            output += f'\n{indent}1. **{dir_name}**\n\n    ---\n\n'  # Add the directory to the TOC with indentation
            include_dir_name_flag = True
            output += generate_toc(subdir, indent + '    ',include_dir_name_flag)
            

        return output    
    output = f'{generate_toc(directory)}'
    return output

print(generate_toc_full('./docs/Recipes'))
print(generate_toc_full('./docs/Recipes/Dessert Options'))


1. **Breads**

    ---

    - [Wheat Gluten Bread](./Breads/recipe_2_wheat_gluten_bread.md)
    - [Tava Roti](./Breads/recipe_1_tava_roti.md)
    - [Tandoori Roti](./Breads/recipe_3_tandoori_roti.md)
    - [Naan Bread](./Breads/recipe_4_naan_bread.md)

1. **Breakfast Options**

    ---

    - [Flaxseed Porridge](./Breakfast%20Options/recipe_001_flaxseed_porridge.md)
    - [Almond Bread](./Breakfast%20Options/recipe_002_almond_bread.md)
    - [Keto Dhokla](./Breakfast%20Options/recipe_003_keto_dhokla.md)
    - [Scrambled Tofu](./Breakfast%20Options/recipe_004_scrambled_tofu.md)
    - [Keto Upma](./Breakfast%20Options/recipe_005_keto_upma.md)

1. **Dessert Options**

    ---

    - [Peanut Mugcake](./Dessert%20Options/recipe_001_peanut_mugcake.md)
    - [Bottlegourd Halwa](./Dessert%20Options/recipe_002_bottlegourd_halwa.md)
    - [Protein Fudge](./Dessert%20Options/recipe_003_protein_fudge.md)
    - [Banana Bread](./Dessert%20Options/recipe_004_banana_bread.md)
    - [Chocolate Cookies

# Generate TOC -- not being used

In [26]:
import os

def generate_toc(directory, indent='    '):
    output = ''

    # Generate TOC for files in the current directory
    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith('.md'):
            output += f'{indent}- {file_name}\n'  # Add the file to the TOC with indentation

    # Recursively generate TOC for subdirectories
    for dir_name in sorted(os.listdir(directory)):
        subdir = os.path.join(directory, dir_name)
        if os.path.isdir(subdir):
            output += f'{indent}- {dir_name}\n'  # Add the directory to the TOC with indentation
            output += generate_toc(subdir, indent + '    ')

    return output

def generate_toc_full(directory):
    output = f'nav:\n{generate_toc(directory)}'
    return output

print(generate_toc_full('./docs/Recipes'))


nav:
    - 00-prepare-to-beat.md
    - index.md
    - Breads
        - index.md
        - recipe_1_tava_roti.md
        - recipe_2_wheat_gluten_bread.md
        - recipe_3_tandoori_roti.md
        - recipe_4_naan_bread.md
    - Breakfast Options
        - index.md
        - recipe_001_flaxseed_porridge.md
        - recipe_002_almond_bread.md
        - recipe_003_keto_dhokla.md
        - recipe_004_scrambled_tofu.md
        - recipe_005_keto_upma.md
    - Dessert Options
        - index.md
        - recipe_001_peanut_mugcake.md
        - recipe_002_bottlegourd_halwa.md
        - recipe_003_protein_fudge.md
        - recipe_004_banana_bread.md
        - recipe_005_chocolate_cookies.md
        - test.md
    - Dips
        - index.md
        - recipe_007_mint_spinach_dip.md
        - recipe_008_green_pesto.md
        - recipe_009_tzatziki_dip.md
        - recipe_012_basil_mint_dip.md
    - Drinks
        - index.md
        - recipe_001_sparkling_cinnamon.md
        - recipe_002_spicy_milk.

# Interactive mode to scrape nutrient info from Sainsbury's - not used

In [28]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Instantiate the Chrome driver
driver = webdriver.Chrome('Downloads/chromedriver_win32/chromedriver.exe')

# Navigate to the Sainsbury's website
driver.get('https://www.sainsburys.co.uk/')

# Wait for the cookie banner to appear
cookie_banner = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, 'onetrust-banner-sdk'))
)

# Find the "Accept All" button and click it
accept_button = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
accept_button.click()
print('clicked')

# Find the search input element by ID
search_input = driver.find_element(By.CSS_SELECTOR, '#term')

# Enter the ingredient value
search_input.send_keys('tomatoes')

# Submit the search form
search_input.submit()
# Wait for the search results to load
search_result = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.si__title'))
)
# Once the results are loaded, you can proceed to extract the nutritional information
# Find the product link and click on it
product_link = driver.find_element(By.CSS_SELECTOR, 'h2.pt__info__description a.pt__link')
product_link.click()

# Wait for the page to load
driver.implicitly_wait(5)

# Extract the nutritional information
nutritional_info = driver.find_element(By.CLASS_NAME, 'nutritionTable')
print(nutritional_info.text)


# Clean up - close the browser
driver.quit()


C:\Users\mittala\AppData\Local\Temp\ipykernel_65108\3223830242.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('Downloads/chromedriver_win32/chromedriver.exe')


clicked


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"h2.pt__info__description a.pt__link"}
  (Session info: chrome=114.0.5735.135)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0045A813+48355]
	(No symbol) [0x003EC4B1]
	(No symbol) [0x002F5358]
	(No symbol) [0x003209A5]
	(No symbol) [0x00320B3B]
	(No symbol) [0x0034E232]
	(No symbol) [0x0033A784]
	(No symbol) [0x0034C922]
	(No symbol) [0x0033A536]
	(No symbol) [0x003182DC]
	(No symbol) [0x003193DD]
	GetHandleVerifier [0x006BAABD+2539405]
	GetHandleVerifier [0x006FA78F+2800735]
	GetHandleVerifier [0x006F456C+2775612]
	GetHandleVerifier [0x004E51E0+616112]
	(No symbol) [0x003F5F8C]
	(No symbol) [0x003F2328]
	(No symbol) [0x003F240B]
	(No symbol) [0x003E4FF7]
	BaseThreadInitThunk [0x753B00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77407B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77407B1E+238]


In [ ]:
def get_nutritional_info(ingredients):
    # Instantiate the Chrome driver
    driver = webdriver.Chrome('Downloads/chromedriver_win32/chromedriver.exe')

    # Navigate to the Sainsbury's website
    driver.get('https://www.sainsburys.co.uk/')

    # Wait for the cookie banner to appear
    cookie_banner = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, 'onetrust-banner-sdk'))
    )

    # Find the "Accept All" button and click it
    accept_button = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
    accept_button.click()
    print('Clicked on Accept All')

    for ingredient in ingredients:
        # Find the search input element by ID
        search_input = driver.find_element(By.CSS_SELECTOR, '#term')

        # Enter the ingredient value
        search_input.clear()
        search_input.send_keys(ingredient)

        # Submit the search form
        search_input.submit()

        # Wait for the search results to load
        search_result = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, 'h1.si__title'))
        )

        # Once the results are loaded, you can proceed to extract the nutritional information
        # Find the product link and click on it
        product_link = driver.find_element(By.CSS_SELECTOR, 'h2.pt__info__description a.pt__link')
        product_link.click()

        # Wait for the page to load
        driver.implicitly_wait(5)

        # Extract the nutritional information
        nutritional_info = driver.find_element(By.CLASS_NAME, 'nutritionTable')
        print(f'Nutritional Information for {ingredient}:\n{nutritional_info.text}\n')

        # Go back to the homepage
        driver.get('https://www.sainsburys.co.uk/')

    # Clean up - close the browser
    driver.quit()


In [ ]:
ingredients = ['tomatoes', 'apples', 'bananas', 'Paneer', 'almonds']
get_nutritional_info(ingredients)